# User keyword query

    Input 1: user query keywords
    Input 2: condition--and, or

    Output: frequency of the keywords
    (1): How many times are the keywords mentioned? 這個關鍵字被提到多少次?
    (2): How many pieces of news contain (mention) the keywords?  有幾篇新聞提到這個關鍵字?

    First article: ['肺炎','疫情', '肺炎']
    Second article:['陳時中','指揮中心','肺炎','陳時中']

    肺炎: 
    (1) '肺炎' are mentioned three times.  ==> frequency is 3
    (2) Two pieces of news mention '肺炎'. ==> occurrence is 2

    陳時中: 
    (1) '陳時中' are mentioned two times.  ==> frequency is 2
    (2) One pieces of news mention '陳時中'. ==> occurrence is 1
    


# Step 0: Load preprocessed news dataset

In [17]:
import pandas as pd
from datetime import datetime, timedelta

In [18]:
df = pd.read_csv('./cna_news_preprocessed.csv',sep='|')

In [19]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,aipl_20220314_1,2022-03-14,政治,外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾,民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中...,0.01,"['外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外', '親赴外交部捐贈物資的民眾約173...","[('外交部', 14), ('民眾', 7), ('物資', 7), ('烏克蘭', 5)...","['民眾', '捐贈', '烏克蘭', '的', '愛心', '物資', '持續', '湧入...","['民眾', '烏克蘭', '愛心', '物資', '外交部', '收到', '外交部長',...","[NerToken(word='烏克蘭', ner='GPE', idx=(4, 7)), ...","[('民眾', 'Na'), ('捐贈', 'VD'), ('烏克蘭', 'Nc'), ('...",https://www.cna.com.tw/news/aipl/202203140364....,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...


# Step 1: Select news with user input keywords, category, duration

## (1)Improved Search from "content" column

In [20]:
from datetime import datetime, timedelta
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_dataFrame(user_keywords, cond, cate, weeks):#關鍵字、and、or、分類、週數

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: keywords 
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in user_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in user_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


## (2)Search keywords from "content" column (Another way)

In [21]:
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_dataFrame_fullText_v0(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # proceed filtering
    if (cate == "全部") & (cond == 'and'):
        df_query = df[(df.date >= start_date) & (df.date <= end_date) 
            & df.content.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cate == "全部") & (cond == 'or'):
        df_query = df[(df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.content.apply(lambda text: any((qk in text) for qk in user_keywords))]
    elif (cond == 'and'):
        df_query = df[(df.category == cate) 
            & (df.date >= start_date) & (df.date <= end_date) 
            & df.content.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cond == 'or'):
        df_query = df[(df.category == cate) 
            & (df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.content.apply(lambda text: any((qk in text) for qk in user_keywords))]

    return df_query


## (3) Search from token_v2 (Another way)

In [22]:
# Searching keywords from "token_v2" column
def filter_dataFrame_v0(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # proceed filtering
    if (cate == "全部") & (cond == 'and'):
        query_df = df[(df.date >= start_date) & (df.date <= end_date) 
            & df.tokens_v2.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cate == "全部") & (cond == 'or'):
        query_df = df[(df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.tokens_v2.apply(lambda text: any((qk in text) for qk in user_keywords))]
    elif (cond == 'and'):
        query_df = df[(df.category == cate) 
            & (df.date >= start_date) & (df.date <= end_date) 
            & df.tokens_v2.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cond == 'or'):
        query_df = df[(df.category == cate) 
            & (df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.tokens_v2.apply(lambda text: any((qk in text) for qk in user_keywords))]
    return query_df

In [23]:
user_keywords=['烏克蘭','俄羅斯']
cond='and'
cate='全部'
weeks=4
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)
len(df_query)

17

In [24]:
user_keywords=['烏克蘭','俄羅斯']
cond='and'
cate='國際'
weeks=4
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)
len(df_query)

8

In [25]:
user_keywords=['烏克蘭','俄羅斯']
cond='or'
cate='政治'
weeks=4
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)
len(df_query)

3

In [26]:
df_query.head()

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,aipl_20220314_1,2022-03-14,政治,外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾,民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中...,0.01,"['外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外', '親赴外交部捐贈物資的民眾約173...","[('外交部', 14), ('民眾', 7), ('物資', 7), ('烏克蘭', 5)...","['民眾', '捐贈', '烏克蘭', '的', '愛心', '物資', '持續', '湧入...","['民眾', '烏克蘭', '愛心', '物資', '外交部', '收到', '外交部長',...","[NerToken(word='烏克蘭', ner='GPE', idx=(4, 7)), ...","[('民眾', 'Na'), ('捐贈', 'VD'), ('烏克蘭', 'Nc'), ('...",https://www.cna.com.tw/news/aipl/202203140364....,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
11,aipl_20220314_12,2022-03-14,政治,賑濟烏克蘭善款 累計突破6億元,財團法人賑災基金會今天表示，賑濟烏克蘭專戶開設到今天下午4時10分，累計收到善款已突破新台幣...,0.00,"['收到紐約台灣商會捐入一筆3萬5703美元、折合新台幣約101萬元的捐款', '外交部2日...","[('基金會', 6), ('烏克蘭', 6), ('收到', 4), ('新台幣', 4)...","['財團', '法人', '賑災', '基金會', '今天', '表示', '，', '賑濟...","['財團', '法人', '賑災', '基金會', '賑濟', '烏克蘭', '專戶', '...","[NerToken(word='今天', ner='DATE', idx=(9, 11)),...","[('財團', 'Na'), ('法人', 'Na'), ('賑災', 'VA'), ('基...",https://www.cna.com.tw/news/aipl/202203140241....,https://imgcdn.cna.com.tw/www/webphotos/WebCov...
19,aipl_20220314_20,2022-03-14,政治,徐國勇：個人贊成延長役期 花點時間讓國家永續安全,烏克蘭情勢讓台灣4個月軍事訓練役的討論浮上檯面。內政部長徐國勇今天說，這牽涉國家兵役政策，應...,0.00,"['徐國勇答詢時說', '內政部長徐國勇今天說', '「一年、幾個月（役期）時間又算什麼',...","[('徐國勇', 6), ('國家', 6), ('戰爭', 4), ('延長', 3), ...","['烏克蘭', '情勢', '讓', '台灣', '4', '個', '月', '軍事', ...","['烏克蘭', '情勢', '台灣', '軍事', '訓練', '檯面', '內政部長', ...","[NerToken(word='烏克蘭', ner='GPE', idx=(0, 3)), ...","[('烏克蘭', 'Nc'), ('情勢', 'Na'), ('讓', 'VL'), ('台...",https://www.cna.com.tw/news/aipl/202203140174....,NaN


# Step 2: calculate frequency and occurence

In [27]:
# For the query_df, count the occurence and frequency for each category.

# (1) cate_occurence={}  被多少篇新聞報導 How many pieces of news contain the keywords.
# (2) cate_freq={}       被提到多少次? How many times are the keywords mentioned


news_categories = ['政治', '科技', '運動', '證卷', '產經', '娛樂', '生活', '國際', '社會', '文化', '兩岸', '全部']

def count_keyword(query_df, user_keywords):
    cate_occurence={}
    cate_freq={}

    for cate in news_categories:
        cate_occurence[cate]=0
        cate_freq[cate]=0

    for idx, row in query_df.iterrows():
        # count number of news
        cate_occurence[row.category] += 1
        cate_occurence['全部'] += 1
        
        # count user keyword frequency by checking every word in tokens_v2
        tokens = eval(row.tokens_v2)
        freq =  len([word for word in tokens if (word in user_keywords)])
        cate_freq[row.category] += freq
        cate_freq['全部'] += freq
        
    return cate_freq, cate_occurence

In [28]:
user_keywords=['烏克蘭','俄羅斯']
cond='or'
cate='全部'
weeks=4
# Step 1 fitering data
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)
len(df_query)

# Step 2: calculating frequency and occurence
count_keyword(df_query, user_keywords)

({'政治': 13,
  '科技': 21,
  '運動': 3,
  '證卷': 2,
  '產經': 15,
  '娛樂': 4,
  '生活': 0,
  '國際': 46,
  '社會': 0,
  '文化': 14,
  '兩岸': 24,
  '全部': 142},
 {'政治': 3,
  '科技': 2,
  '運動': 1,
  '證卷': 2,
  '產經': 3,
  '娛樂': 2,
  '生活': 0,
  '國際': 8,
  '社會': 0,
  '文化': 3,
  '兩岸': 2,
  '全部': 26})

In [29]:
len(df_query)

26

# Demonstrate step by step

How many news are related to "烏克蘭" ?

    How many pieces of news mentioned "烏克蘭"
    How many pieces of news are related to "烏克蘭"?

    You can calculate and get the answer from the following fields: tokens, tokens_v2, or content. (Get very similar results)

    We use "tokens_v2" because it contains only some important keywords which were selected in the pre-process step.

    
A flexible appraoch for And OR condiction

        User all()  any()
        df = pd.DataFrame({'col': ["apple is delicious",
                                "banana is delicious",
                                "apple and banana both are delicious"]})

        targets = ['apple', 'banana']

        # Any word from `targets` are present in sentence.
        >>> df.col.apply(lambda sentence: any(word in sentence for word in targets))
        0    True
        1    True
        2    True
        Name: col, dtype: bool

        # All words from `targets` are present in sentence.
        >>> df.col.apply(lambda sentence: all(word in sentence for word in targets))
        0    False
        1    False
        2     True
        Name: col, dtype: bool

## and &, or | 

In [30]:
True & True

True

In [31]:
True & False

False

In [32]:
True | True

True

In [33]:
True | False

True

## "in" is very powerful in Python!

#### in a string

In [34]:
text = '武漢烏克蘭疫情全球延燒，國防部2月針對29個疫情高風險國家地區勸阻官兵前往（包括過境）。'

In [35]:
'勸阻官兵' in text

True

In [36]:
'延燒，國防部' in text

True

In [37]:
'台灣' in text

False

In [38]:
'烏克蘭' in text

True

In [39]:
# & and
('台灣' in text)  & ('烏克蘭' in text)

False

In [40]:
# & and
('台灣' in text) and ('烏克蘭' in text)

False

In [41]:
('台灣' in text)  | ('烏克蘭' in text)

True

In [42]:
('台灣' in text)  or ('烏克蘭' in text)

True

In [43]:
# This is also a string.
text = "['武漢', '烏克蘭', '疫情', '全球', '延燒', '國防部', '疫情', '高風險', '國家', '地區', '官兵', '過境', '國防部', '政策', '全球', '國家', '地區', '轄下', '單位']"

In [44]:
'烏克蘭' in text

True

In [45]:
'台灣' in text

False

In [46]:
('台灣' in text)  & ('烏克蘭' in text)

False

In [47]:
('台灣' in text)  | ('烏克蘭' in text)

True

#### in a list

In [48]:
user_keyword=['烏克蘭','台灣']
'烏克蘭' in user_keyword

True

In [49]:
# Check out the first news
df.content[0]

'民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝。外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日。外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問。根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜。外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物。送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾。募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部。外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力。外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意。'

In [50]:
qk = '烏克蘭'
text = df.content[0]
qk in text

True

In [51]:
qk = '外交部'
text = df.content[0]
qk in text

True

In [52]:
qk = '台灣'
text = df.content[0]
qk in text

False

In [53]:
text = df.content[0]
('烏克蘭' in text) & ('外交部' in text )

True

### Another "in" in Python. It is used for "for" loop.

In [54]:
user_keywords = ['烏克蘭','外交部']
text = df.content[0]
[(qk in text) for qk in user_keywords]

[True, True]

In [55]:
user_keywords = ['烏克蘭','台灣']
text = df.content[0]
[(qk in text) for qk in user_keywords]

[True, False]

### all() any() 

    How to perform logical opertion with several conditions? 如何針對很多項去做邏輯運算?

    all(): perform "and" logical opertion 
    any(): perform "or" logical opertion

In [56]:
all( [True, True, True] ) #　True & True

True

In [57]:
any( [True, True] )

True

In [58]:
all( [True, False] )

False

In [59]:
any( [True, False] )

True

In [60]:
user_keywords = ['烏克蘭','國防部']
[word for word in user_keywords]

['烏克蘭', '國防部']

In [61]:
user_keywords=['烏克蘭','國防部']
text = '武漢烏克蘭疫情全球延燒，國防部2月針對29個疫情高風險國家地區勸阻官兵前往（包括過境）。'
[(word in text) for word in user_keywords]

[True, True]

In [62]:
user_keywords=['烏克蘭','國防部']
text = '武漢烏克蘭疫情全球延燒，國防部2月針對29個疫情高風險國家地區勸阻官兵前往（包括過境）。'
[word in text for word in user_keywords] # () can be removed

[True, True]

In [63]:
user_keywords=['烏克蘭','國防部']
text = '武漢烏克蘭疫情全球延燒，國防部2月針對29個疫情高風險國家地區勸阻官兵前往（包括過境）。'
all([word in text for word in user_keywords])

True

In [64]:
user_keywords=['烏克蘭','國防部']
text = '武漢烏克蘭疫情全球延燒，國防部2月針對29個疫情高風險國家地區勸阻官兵前往（包括過境）。'
all(word in text for word in user_keywords) # square brackets [] can be removed

True

In [65]:
# Check out the first news
user_keywords = ['烏克蘭','外交部']
text = df.content[0]
print([(qk in text) for qk in user_keywords])
all((qk in text) for qk in user_keywords)

[True, True]


True

In [66]:
user_keywords = ['烏克蘭','外交部']
text = df.content[0]
any((qk in text) for qk in user_keywords)

True

In [67]:
user_keywords = ['烏克蘭','台灣']
text = df.content[0]
[(qk in text) for qk in user_keywords]

[True, False]

In [68]:
user_keywords = ['烏克蘭','台灣']
text = df.content[0]
all((qk in text) for qk in user_keywords)

False

In [69]:
user_keywords = ['烏克蘭','台灣']
text = df.content[0]
any((qk in text) for qk in user_keywords)

True

## Using apply() and lambda function

How to check out keyword occurency for every news?

In [70]:
# Use apply() and lambda function
user_keywords = ['烏克蘭','外交部']
df.content.apply(lambda text: all([(qk in text) for qk in user_keywords]))

0       True
1      False
2      False
3      False
4      False
       ...  
208    False
209    False
210    False
211    False
212    False
Name: content, Length: 213, dtype: bool

In [71]:
user_keywords = ['烏克蘭','外交部']

In [72]:
[qk for qk in user_keywords]

['烏克蘭', '外交部']

In [73]:
text = df.content[0]
text

'民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝。外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日。外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問。根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜。外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物。送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾。募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部。外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力。外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意。'

In [74]:
[(qk in text) for qk in user_keywords]

[True, True]

In [75]:
# 
all([(qk in text) for qk in user_keywords])

True

In [76]:
# Square brackets can be removed
all((qk in text) for qk in user_keywords)

True

In [77]:
df[df.tokens_v2.apply(lambda text: all([word in text for word in user_keyword]))]

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
11,aipl_20220314_12,2022-03-14,政治,賑濟烏克蘭善款 累計突破6億元,財團法人賑災基金會今天表示，賑濟烏克蘭專戶開設到今天下午4時10分，累計收到善款已突破新台幣...,0.0,"['收到紐約台灣商會捐入一筆3萬5703美元、折合新台幣約101萬元的捐款', '外交部2日...","[('基金會', 6), ('烏克蘭', 6), ('收到', 4), ('新台幣', 4)...","['財團', '法人', '賑災', '基金會', '今天', '表示', '，', '賑濟...","['財團', '法人', '賑災', '基金會', '賑濟', '烏克蘭', '專戶', '...","[NerToken(word='今天', ner='DATE', idx=(9, 11)),...","[('財團', 'Na'), ('法人', 'Na'), ('賑災', 'VA'), ('基...",https://www.cna.com.tw/news/aipl/202203140241....,https://imgcdn.cna.com.tw/www/webphotos/WebCov...
19,aipl_20220314_20,2022-03-14,政治,徐國勇：個人贊成延長役期 花點時間讓國家永續安全,烏克蘭情勢讓台灣4個月軍事訓練役的討論浮上檯面。內政部長徐國勇今天說，這牽涉國家兵役政策，應...,0.0,"['徐國勇答詢時說', '內政部長徐國勇今天說', '「一年、幾個月（役期）時間又算什麼',...","[('徐國勇', 6), ('國家', 6), ('戰爭', 4), ('延長', 3), ...","['烏克蘭', '情勢', '讓', '台灣', '4', '個', '月', '軍事', ...","['烏克蘭', '情勢', '台灣', '軍事', '訓練', '檯面', '內政部長', ...","[NerToken(word='烏克蘭', ner='GPE', idx=(0, 3)), ...","[('烏克蘭', 'Nc'), ('情勢', 'Na'), ('讓', 'VL'), ('台...",https://www.cna.com.tw/news/aipl/202203140174....,NaN
33,ait_20220309_14,2022-03-09,科技,Apple新品一次看 iPhone SE售1萬3900元起iPad Air有5色[影],蘋果公司（Apple）今天舉行線上春季發表會，平價版的iPhone SE和iPad Air將...,0.0,"['新iPad Air搭載iPad Pro上的M1晶片', 'iPad Air則搭載蘋果為筆...","[('蘋果', 15), ('晶片', 15), ('搭載', 10), ('售價', 8)...","['蘋果', '公司', '（', 'Apple', '）', '今天', '舉行', '線...","['蘋果', '公司', '舉行', '線上', '發表會', '平價版', '支援', '...","[NerToken(word='今天', ner='DATE', idx=(11, 13))...","[('蘋果', 'Na'), ('公司', 'Nc'), ('（', 'PARENTHESI...",https://www.cna.com.tw/news/ait/202203090006.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...
61,asc_20220314_2,2022-03-14,證卷,俄烏戰火不止 法人：台股短線恐震盪整理,俄烏戰火持續影響全球股市，法人表示，在俄烏停火前，短線台股震盪整理機會大；但由於台股基本面具...,0.0,"['不會改變股市的長期走勢', '台股本週觀察重點在於美國聯準會公開市場操作委員會（FOMC...","[('台股', 9), ('市場', 6), ('俄烏', 4), ('影響', 4), (...","['俄烏', '戰火', '持續', '影響', '全球', '股市', '，', '法人'...","['俄烏', '戰火', '影響', '全球', '股市', '法人', '俄烏', '短線...","[NerToken(word='烏', ner='GPE', idx=(20, 21)), ...","[('俄烏', 'Nc'), ('戰火', 'Na'), ('持續', 'VL'), ('影...",https://www.cna.com.tw/news/afe/202203140355.aspx,NaN
98,aie_20220314_19,2022-03-14,產經,油價若續漲 朱澤民：今年CPI有可能超過2%,立委質詢關切俄烏戰事對台灣經濟與民生影響，行政院主計總處主計長朱澤民今天指出，原本估計第1季...,0.0,"['立委質詢關切俄烏戰事對台灣經濟與民生影響', '國民黨立委林德福擔心國際油價上漲會衝擊民...","[('朱澤民', 9), ('油價', 7), ('影響', 5), ('立委', 4), ...","['立委', '質詢', '關切', '俄烏', '戰事', '對', '台灣', '經濟'...","['立委', '俄烏', '戰事', '台灣', '經濟', '民生', '影響', '行政...","[NerToken(word='台灣', ner='GPE', idx=(11, 13)),...","[('立委', 'Na'), ('質詢', 'VE'), ('關切', 'VJ'), ('俄...",https://www.cna.com.tw/news/afe/202203140204.aspx,NaN
149,aopl_20220314_10,2022-03-14,國際,路透社：美中高層已在羅馬會晤,路透社引述中國中央電視台報導，中共中央外事工作委員會辦公室主任楊潔篪今天在義大利羅馬，與白宮...,0.0,"['蘇利文當時在會中也表達美方對中國在人權、新疆、香港、南海以及台灣行動的關切', '也將討...","[('會晤', 7), ('蘇利文', 5), ('中國', 4), ('楊潔篪', 4),...","['路透社', '引述', '中國', '中央', '電視台', '報導', '，', '中...","['路透社', '引述', '中國', '中央', '電視台', '中共', '中央', '...","[NerToken(word='路透社', ner='ORG', idx=(0, 3)), ...","[('路透社', 'Nc'), ('引述', 'VC'), ('中國', 'Nc'), ('...",https://www.cna.com.tw/news/aopl/202203140309....,https://imgcdn.cna.com.tw/www/webphotos/WebCov...
179,acul_20220314_7,2022-03-14,文化,國際博物館協會發聲拒絕戰爭 吳思瑤籲故宮跟進,俄羅斯入侵烏克蘭戰事未歇，國際博物館協會（ICOM）呼籲應拒絕戰爭並搶救文物。立委吳思瑤今天...,0.0,"['民進黨籍立委吳思瑤以國際博物館協會於2月24日針對烏克蘭戰事發布的聲明指出', '故宮同...","[('博物館', 9), ('烏克蘭', 8), ('故宮', 8), ('國際', 5),...","['俄羅斯', '入侵', '烏克蘭', '戰事', '未', '歇', '，', '國際'...","['俄羅斯', '烏克蘭', '戰事', '國際', '博物館', '協會', '戰爭', ...","[NerToken(word='俄羅斯', ner='GPE', idx=(0, 3)), ...","[('俄羅斯', 'Nc'), ('入侵', 'VCL'), ('烏克蘭', 'Nc'), ...",https://www.cna.com.tw/news/acul/202203140198....,NaN


In [78]:
len(df[df.tokens_v2.apply(lambda text: all([word in text for word in user_keyword]))])

7

### lambda: how does it work?
lambda is a function with only one line

In [79]:
func = lambda x: x+5  # parameters函數的參數   func(55, 175)  func(x) -> x+5

In [80]:
func(2)

7

In [81]:
numbers=[1,2,3]

In [82]:
[func(x) for x in numbers]

[6, 7, 8]

In [83]:
numbers=[1,2,3]
list(map(func, numbers))

[6, 7, 8]

In [84]:
numbers=[1,2,3]
list(map(lambda x: x+5, numbers))

[6, 7, 8]

In [85]:
def func2(x):
    return x+10

numbers=[1,2,3]
list(map(func2, numbers))

[11, 12, 13]

### How to use pandas apply()?

In [86]:
df_test = pd.DataFrame([1,2,3], columns=['number'])

In [87]:
df_test

,number
0,1
1,2
2,3


In [88]:
df_test.number.apply(lambda x: x+5) # add 5 for each element in df_test.number

0    6
1    7
2    8
Name: number, dtype: int64

In [89]:
df.tokens_v2.apply(lambda text: all([word in text for word in user_keyword]))

0      False
1      False
2      False
3      False
4      False
       ...  
208    False
209    False
210    False
211    False
212    False
Name: tokens_v2, Length: 213, dtype: bool

In [90]:
df.content.apply(lambda text: all([word in text for word in user_keyword]))

0      False
1      False
2      False
3      False
4      False
       ...  
208    False
209    False
210    False
211    False
212    False
Name: content, Length: 213, dtype: bool

In [91]:
user_keywords = ['烏克蘭','外交部']
def func3(text): 
    return all([word in text for word in user_keyword])

df.content.apply(func3)

0      False
1      False
2      False
3      False
4      False
       ...  
208    False
209    False
210    False
211    False
212    False
Name: content, Length: 213, dtype: bool

# Function of fitering news

Put them all together!

In [92]:
# Searching keywords from "token_v2" column
def filter_dataFrame(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # proceed filtering
    if (cate == "全部") & (cond == 'and'):
        query_df = df[(df.date >= start_date) & (df.date <= end_date) 
            & df.tokens_v2.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cate == "全部") & (cond == 'or'):
        query_df = df[(df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.tokens_v2.apply(lambda text: any((qk in text) for qk in user_keywords))]
    elif (cond == 'and'):
        query_df = df[(df.category == cate) 
            & (df.date >= start_date) & (df.date <= end_date) 
            & df.tokens_v2.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cond == 'or'):
        query_df = df[(df.category == cate) 
            & (df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.tokens_v2.apply(lambda text: any((qk in text) for qk in user_keywords))]
    return query_df

### For "politics" category 政治類

In [93]:
user_keywords=['烏克蘭','台灣']
cond='and'
cate='政治'
weeks=2
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)

In [94]:
# end date: the date of the latest record of news
end_date = df.date.max()

# start date
start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

In [95]:
user_keywords=['烏克蘭','台灣']
cond='and'
cate='政治'
weeks=2

In [96]:
(df.category == cate) & ( (df['date'] >= start_date) & (df['date'] <= end_date)  ) & df.tokens_v2.apply(lambda text: any((qk in text) for qk in user_keywords))

0       True
1      False
2      False
3      False
4       True
       ...  
208    False
209    False
210    False
211    False
212    False
Length: 213, dtype: bool

In [97]:
query_df = df[(
    df.category == cate)   # category
    & (df.date >= start_date) & (df.date <= end_date) # duration
    & df.tokens_v2.apply(lambda row: all((qk in row) for qk in user_keywords)) # user keywords
    ]
len(query_df)

2

In [98]:
query_df

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
11,aipl_20220314_12,2022-03-14,政治,賑濟烏克蘭善款 累計突破6億元,財團法人賑災基金會今天表示，賑濟烏克蘭專戶開設到今天下午4時10分，累計收到善款已突破新台幣...,0.0,"['收到紐約台灣商會捐入一筆3萬5703美元、折合新台幣約101萬元的捐款', '外交部2日...","[('基金會', 6), ('烏克蘭', 6), ('收到', 4), ('新台幣', 4)...","['財團', '法人', '賑災', '基金會', '今天', '表示', '，', '賑濟...","['財團', '法人', '賑災', '基金會', '賑濟', '烏克蘭', '專戶', '...","[NerToken(word='今天', ner='DATE', idx=(9, 11)),...","[('財團', 'Na'), ('法人', 'Na'), ('賑災', 'VA'), ('基...",https://www.cna.com.tw/news/aipl/202203140241....,https://imgcdn.cna.com.tw/www/webphotos/WebCov...
19,aipl_20220314_20,2022-03-14,政治,徐國勇：個人贊成延長役期 花點時間讓國家永續安全,烏克蘭情勢讓台灣4個月軍事訓練役的討論浮上檯面。內政部長徐國勇今天說，這牽涉國家兵役政策，應...,0.0,"['徐國勇答詢時說', '內政部長徐國勇今天說', '「一年、幾個月（役期）時間又算什麼',...","[('徐國勇', 6), ('國家', 6), ('戰爭', 4), ('延長', 3), ...","['烏克蘭', '情勢', '讓', '台灣', '4', '個', '月', '軍事', ...","['烏克蘭', '情勢', '台灣', '軍事', '訓練', '檯面', '內政部長', ...","[NerToken(word='烏克蘭', ner='GPE', idx=(0, 3)), ...","[('烏克蘭', 'Nc'), ('情勢', 'Na'), ('讓', 'VL'), ('台...",https://www.cna.com.tw/news/aipl/202203140174....,NaN


### For "全部""All" category (不必篩選類別!)

In [99]:
user_keywords=['烏克蘭','台灣']
cond='and'
cate='全部'
weeks=2

In [100]:
if (cate == "全部") & (cond == 'and'):
    query_df = df[
        (df.date >= start_date) & (df.date <= end_date) 
        & df.tokens_v2.apply(lambda row: all((qk in row) for qk in user_keywords))
        ]
elif (cate == "全部") & (cond == 'or'):
    query_df = df[
        (df['date'] >= start_date) & (df['date'] <= end_date) 
        & df.tokens_v2.apply(lambda row: any((qk in row) for qk in user_keywords))]

In [101]:
len(query_df)

7

# Function of count_keyword() to calculate frequency and occurence

In [102]:
# For the query_df, count the occurence and frequency for each category.

# (1) cate_occurence={}  被多少篇新聞報導
# (2) cate_freq={}       被提到多少次?How many times were the keywords mentioned


news_categories = ['政治', '科技', '運動', '證卷', '產經', '娛樂', '生活', '國際', '社會', '文化', '兩岸', '全部']

def count_keyword(query_df, user_keywords):
    cate_occurence={}
    cate_freq={}

    for cate in news_categories:
        cate_occurence[cate]=0
        cate_freq[cate]=0

    for idx, row in query_df.iterrows():
        # count number of news
        cate_occurence[row.category] += 1 # add 1 to its category's occurence
        cate_occurence['全部'] += 1
        
        # count user keyword frequency by checking every word in tokens_v2
        tokens = eval(row.tokens_v2) # count frequency
        freq =  len([word for word in tokens if (word in user_keywords)]) # how manay times?
        cate_freq[row.category] += freq
        cate_freq['全部'] += freq
        
    return cate_freq, cate_occurence

In [103]:
user_keywords=['烏克蘭','台灣']
count_keyword(df_query, user_keywords)

({'政治': 14,
  '科技': 0,
  '運動': 0,
  '證卷': 0,
  '產經': 0,
  '娛樂': 0,
  '生活': 0,
  '國際': 0,
  '社會': 0,
  '文化': 0,
  '兩岸': 0,
  '全部': 14},
 {'政治': 2,
  '科技': 0,
  '運動': 0,
  '證卷': 0,
  '產經': 0,
  '娛樂': 0,
  '生活': 0,
  '國際': 0,
  '社會': 0,
  '文化': 0,
  '兩岸': 0,
  '全部': 2})

## Show it step by step by yourself

In [104]:
user_keywords=['烏克蘭','外交部']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)

# count user keyword frequency by checking every word in tokens_v2
tokens = eval(df_query.tokens_v2[0]) # count frequency
freq =  len([word for word in tokens if (word in user_keywords)]) # how manay times?

In [105]:
df_query

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,aipl_20220314_1,2022-03-14,政治,外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾,民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中...,0.01,"['外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外', '親赴外交部捐贈物資的民眾約173...","[('外交部', 14), ('民眾', 7), ('物資', 7), ('烏克蘭', 5)...","['民眾', '捐贈', '烏克蘭', '的', '愛心', '物資', '持續', '湧入...","['民眾', '烏克蘭', '愛心', '物資', '外交部', '收到', '外交部長',...","[NerToken(word='烏克蘭', ner='GPE', idx=(4, 7)), ...","[('民眾', 'Na'), ('捐贈', 'VD'), ('烏克蘭', 'Nc'), ('...",https://www.cna.com.tw/news/aipl/202203140364....,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
11,aipl_20220314_12,2022-03-14,政治,賑濟烏克蘭善款 累計突破6億元,財團法人賑災基金會今天表示，賑濟烏克蘭專戶開設到今天下午4時10分，累計收到善款已突破新台幣...,0.00,"['收到紐約台灣商會捐入一筆3萬5703美元、折合新台幣約101萬元的捐款', '外交部2日...","[('基金會', 6), ('烏克蘭', 6), ('收到', 4), ('新台幣', 4)...","['財團', '法人', '賑災', '基金會', '今天', '表示', '，', '賑濟...","['財團', '法人', '賑災', '基金會', '賑濟', '烏克蘭', '專戶', '...","[NerToken(word='今天', ner='DATE', idx=(9, 11)),...","[('財團', 'Na'), ('法人', 'Na'), ('賑災', 'VA'), ('基...",https://www.cna.com.tw/news/aipl/202203140241....,https://imgcdn.cna.com.tw/www/webphotos/WebCov...
140,aopl_20220314_1,2022-03-14,國際,澳洲擴大制裁俄羅斯 歐盟要凍結切爾西老闆資產,（中央社雪梨/布魯塞爾14日綜合外電報導）澳洲為懲罰俄羅斯侵略烏克蘭，今天再將33名俄國寡頭...,0.00,"['將阿布拉莫維奇加入制裁的俄羅斯富豪名單', '而歐盟各國代表將在今天開會通過這項措施',...","[('制裁', 8), ('俄羅斯', 7), ('俄國', 7), ('歐盟', 5), ...","['（', '中央社', '雪梨', '/', '布魯塞爾', '14日', '綜合', '...","['中央社', '雪梨', '布魯塞爾', '綜合', '外電', '澳洲', '懲罰', ...","[NerToken(word='中央社', ner='ORG', idx=(1, 4)), ...","[('（', 'PARENTHESISCATEGORY'), ('中央社', 'Nc'), ...",https://www.cna.com.tw/news/aopl/202203140376....,NaN
141,aopl_20220314_2,2022-03-14,國際,俄羅斯提核協議新要求 伊朗外長將赴莫斯科討論,關於伊朗核子協議的談判，數天前因為俄羅斯提出新要求而觸礁，伊朗外交部長阿布杜拉希安15日將赴...,0.00,['他已與俄羅斯外長拉夫羅夫（Sergei Lavrov）討論了正在奧地利維也納進行的伊朗核...,"[('伊朗', 11), ('協議', 9), ('談判', 6), ('外長', 6), ...","['關於', '伊朗', '核子', '協議', '的', '談判', '，', '數', ...","['伊朗', '核子', '協議', '談判', '俄羅斯', '提出', '要求', '伊...","[NerToken(word='伊朗', ner='GPE', idx=(2, 4)), N...","[('關於', 'P'), ('伊朗', 'Nc'), ('核子', 'Na'), ('協議...",https://www.cna.com.tw/news/aopl/202203140361....,NaN


In [106]:
df_query.iloc[0]

item_id                                           aipl_20220314_1
date                                                   2022-03-14
category                                                       政治
title                                    外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾
content         民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中...
sentiment                                                    0.01
summary         ['外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外', '親赴外交部捐贈物資的民眾約173...
top_key_freq    [('外交部', 14), ('民眾', 7), ('物資', 7), ('烏克蘭', 5)...
tokens          ['民眾', '捐贈', '烏克蘭', '的', '愛心', '物資', '持續', '湧入...
tokens_v2       ['民眾', '烏克蘭', '愛心', '物資', '外交部', '收到', '外交部長',...
entities        [NerToken(word='烏克蘭', ner='GPE', idx=(4, 7)), ...
token_pos       [('民眾', 'Na'), ('捐贈', 'VD'), ('烏克蘭', 'Nc'), ('...
link            https://www.cna.com.tw/news/aipl/202203140364....
photo_link      https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
Name: 0, dtype: object

In [107]:
row = df_query.iloc[0]

In [108]:
row.tokens_v2

"['民眾', '烏克蘭', '愛心', '物資', '外交部', '收到', '外交部長', '吳釗燮', '現場', '協助', '整理', '物資', '志工', '加油', '民眾', '表達', '外交部', '發布', '新聞稿', '外交部', '民間', '募集', '烏克蘭', '難民', '物資', '響應', '外交部', '物資', '民眾', '包裹', '收到', '物資', '品項', '醫療', '口罩', '毛毯', '女性', '衛生', '用品', '尿片', '餅乾', '募集', '活動', '外交部', '民眾', '現場', '分類', '整理', '志工', '外交部', '人員', '吳釗燮', '外交部', '地下', '停車場', '吳釗燮', '在場', '慈濟', '民間', '組織', '志工', '自發', '到場', '幫忙', '善心', '人士', '互動', '烏克蘭', '人士', '義工', '現場', '協助', '吳釗燮', '致意', '慰問', '外交部', '照片', '到場', '幫忙', '烏克蘭', '志工', '家鄉', '情勢', '網紅佳娜', '外交部', '物資', '民眾', '物品', '外交部', '網站', '清單', '物品', '衣物', '外交部', '物品', '物品', '拆封', '包裝', '有效期', '整理', '轉運', '困擾', '時間', '民眾', '清單', '藥品', '外交部', '外交部', '民眾', '援助', '烏克蘭', '人道', '物資', '志工', '幫忙', '貢獻', '外交部', '各界', '人士', '時間', '精神', '投入', '國際', '人道', '援助', '表達', '敬意']"

In [109]:
eval(row.tokens_v2)

['民眾',
 '烏克蘭',
 '愛心',
 '物資',
 '外交部',
 '收到',
 '外交部長',
 '吳釗燮',
 '現場',
 '協助',
 '整理',
 '物資',
 '志工',
 '加油',
 '民眾',
 '表達',
 '外交部',
 '發布',
 '新聞稿',
 '外交部',
 '民間',
 '募集',
 '烏克蘭',
 '難民',
 '物資',
 '響應',
 '外交部',
 '物資',
 '民眾',
 '包裹',
 '收到',
 '物資',
 '品項',
 '醫療',
 '口罩',
 '毛毯',
 '女性',
 '衛生',
 '用品',
 '尿片',
 '餅乾',
 '募集',
 '活動',
 '外交部',
 '民眾',
 '現場',
 '分類',
 '整理',
 '志工',
 '外交部',
 '人員',
 '吳釗燮',
 '外交部',
 '地下',
 '停車場',
 '吳釗燮',
 '在場',
 '慈濟',
 '民間',
 '組織',
 '志工',
 '自發',
 '到場',
 '幫忙',
 '善心',
 '人士',
 '互動',
 '烏克蘭',
 '人士',
 '義工',
 '現場',
 '協助',
 '吳釗燮',
 '致意',
 '慰問',
 '外交部',
 '照片',
 '到場',
 '幫忙',
 '烏克蘭',
 '志工',
 '家鄉',
 '情勢',
 '網紅佳娜',
 '外交部',
 '物資',
 '民眾',
 '物品',
 '外交部',
 '網站',
 '清單',
 '物品',
 '衣物',
 '外交部',
 '物品',
 '物品',
 '拆封',
 '包裝',
 '有效期',
 '整理',
 '轉運',
 '困擾',
 '時間',
 '民眾',
 '清單',
 '藥品',
 '外交部',
 '外交部',
 '民眾',
 '援助',
 '烏克蘭',
 '人道',
 '物資',
 '志工',
 '幫忙',
 '貢獻',
 '外交部',
 '各界',
 '人士',
 '時間',
 '精神',
 '投入',
 '國際',
 '人道',
 '援助',
 '表達',
 '敬意']

In [110]:
tokens

['民眾',
 '烏克蘭',
 '愛心',
 '物資',
 '外交部',
 '收到',
 '外交部長',
 '吳釗燮',
 '現場',
 '協助',
 '整理',
 '物資',
 '志工',
 '加油',
 '民眾',
 '表達',
 '外交部',
 '發布',
 '新聞稿',
 '外交部',
 '民間',
 '募集',
 '烏克蘭',
 '難民',
 '物資',
 '響應',
 '外交部',
 '物資',
 '民眾',
 '包裹',
 '收到',
 '物資',
 '品項',
 '醫療',
 '口罩',
 '毛毯',
 '女性',
 '衛生',
 '用品',
 '尿片',
 '餅乾',
 '募集',
 '活動',
 '外交部',
 '民眾',
 '現場',
 '分類',
 '整理',
 '志工',
 '外交部',
 '人員',
 '吳釗燮',
 '外交部',
 '地下',
 '停車場',
 '吳釗燮',
 '在場',
 '慈濟',
 '民間',
 '組織',
 '志工',
 '自發',
 '到場',
 '幫忙',
 '善心',
 '人士',
 '互動',
 '烏克蘭',
 '人士',
 '義工',
 '現場',
 '協助',
 '吳釗燮',
 '致意',
 '慰問',
 '外交部',
 '照片',
 '到場',
 '幫忙',
 '烏克蘭',
 '志工',
 '家鄉',
 '情勢',
 '網紅佳娜',
 '外交部',
 '物資',
 '民眾',
 '物品',
 '外交部',
 '網站',
 '清單',
 '物品',
 '衣物',
 '外交部',
 '物品',
 '物品',
 '拆封',
 '包裝',
 '有效期',
 '整理',
 '轉運',
 '困擾',
 '時間',
 '民眾',
 '清單',
 '藥品',
 '外交部',
 '外交部',
 '民眾',
 '援助',
 '烏克蘭',
 '人道',
 '物資',
 '志工',
 '幫忙',
 '貢獻',
 '外交部',
 '各界',
 '人士',
 '時間',
 '精神',
 '投入',
 '國際',
 '人道',
 '援助',
 '表達',
 '敬意']

In [111]:
[word for word in tokens if (word in user_keywords)]

['烏克蘭',
 '外交部',
 '外交部',
 '外交部',
 '烏克蘭',
 '外交部',
 '外交部',
 '外交部',
 '外交部',
 '烏克蘭',
 '外交部',
 '烏克蘭',
 '外交部',
 '外交部',
 '外交部',
 '外交部',
 '外交部',
 '烏克蘭',
 '外交部']

In [112]:
len([word for word in tokens if (word in user_keywords)])

19

In [113]:
freq

19

In [114]:
cate_occurence={}
cate_freq={}

In [115]:
cate_occurence

{}

In [116]:
for cate in news_categories:
    cate_occurence[cate]=0
    cate_freq[cate]=0

In [117]:
cate_occurence

{'政治': 0,
 '科技': 0,
 '運動': 0,
 '證卷': 0,
 '產經': 0,
 '娛樂': 0,
 '生活': 0,
 '國際': 0,
 '社會': 0,
 '文化': 0,
 '兩岸': 0,
 '全部': 0}

In [118]:
cate_freq

{'政治': 0,
 '科技': 0,
 '運動': 0,
 '證卷': 0,
 '產經': 0,
 '娛樂': 0,
 '生活': 0,
 '國際': 0,
 '社會': 0,
 '文化': 0,
 '兩岸': 0,
 '全部': 0}

In [119]:
cate_freq[row.category] += freq
cate_freq['全部'] += freq

In [120]:
cate_freq

{'政治': 19,
 '科技': 0,
 '運動': 0,
 '證卷': 0,
 '產經': 0,
 '娛樂': 0,
 '生活': 0,
 '國際': 0,
 '社會': 0,
 '文化': 0,
 '兩岸': 0,
 '全部': 19}

In [121]:
# count number of news
cate_occurence[row.category] += 1 # add 1 to its category's occurence
cate_occurence['全部'] += 1

In [122]:
cate_occurence

{'政治': 1,
 '科技': 0,
 '運動': 0,
 '證卷': 0,
 '產經': 0,
 '娛樂': 0,
 '生活': 0,
 '國際': 0,
 '社會': 0,
 '文化': 0,
 '兩岸': 0,
 '全部': 1}

## Now we can conveniently call those functions

In [123]:
user_keywords=['烏克蘭','台灣']
cond='or'
cate='全部'
weeks=2
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)

In [124]:
len(df_query)

91

In [125]:
count_keyword(df_query, user_keywords)

({'政治': 50,
  '科技': 37,
  '運動': 14,
  '證卷': 27,
  '產經': 34,
  '娛樂': 16,
  '生活': 7,
  '國際': 32,
  '社會': 8,
  '文化': 51,
  '兩岸': 11,
  '全部': 287},
 {'政治': 10,
  '科技': 11,
  '運動': 10,
  '證卷': 7,
  '產經': 9,
  '娛樂': 8,
  '生活': 5,
  '國際': 9,
  '社會': 5,
  '文化': 14,
  '兩岸': 3,
  '全部': 91})

In [126]:
user_keywords=['烏克蘭','台灣']
cond='and'
cate='政治'
weeks=2
df_query = filter_dataFrame(user_keywords, cond, cate, weeks)

In [127]:
len(df_query)

2

In [128]:
count_keyword(df_query, user_keyword)

({'政治': 14,
  '科技': 0,
  '運動': 0,
  '證卷': 0,
  '產經': 0,
  '娛樂': 0,
  '生活': 0,
  '國際': 0,
  '社會': 0,
  '文化': 0,
  '兩岸': 0,
  '全部': 14},
 {'政治': 2,
  '科技': 0,
  '運動': 0,
  '證卷': 0,
  '產經': 0,
  '娛樂': 0,
  '生活': 0,
  '國際': 0,
  '社會': 0,
  '文化': 0,
  '兩岸': 0,
  '全部': 2})

## Usage of apply, map (For reference)

In [129]:
# apply usage
import pandas as pd

sample_df = pd.DataFrame({
    'Col 1': [3,4,5,6],
    'Col 2': [2,3,6,4],
    'Col 3': [8,8,9,8],

},index=["A","B","C","D"])
sample_df

,Col 1,Col 2,Col 3
A,3,2,8
B,4,3,8
C,5,6,9
D,6,4,8


In [130]:
sample_df=sample_df.apply(lambda x: x+10)
sample_df

,Col 1,Col 2,Col 3
A,13,12,18
B,14,13,18
C,15,16,19
D,16,14,18


In [131]:
sample_df["Col 1"]=sample_df["Col 1"].apply(lambda x: x-10)
sample_df

,Col 1,Col 2,Col 3
A,3,12,18
B,4,13,18
C,5,16,19
D,6,14,18
